<a href="https://colab.research.google.com/github/WengGeyu/NMA-DL/blob/main/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q condacolab
import condacolab
condacolab.install_miniconda() # expect a kernel restart

✨🍰✨ Everything looks OK!


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import os
os.chdir('/content/drive/MyDrive/ColabNotebooks/Aneurysm_Detection-main/install')
!conda env update -f environment_clean.yml

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

ERROR: Could not find a version that satisfies the requirement opencv-python==4.5 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.14.51, 3.4.14.53, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.2.52, 4.5.2.54, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72, 4.8.0.74)

ERROR: No matching distribution found for opencv-python==4.5failed


In [ ]:
# !pip install openneuro-py
# import openneuro as on
# on.download(dataset='ds003949', target_dir='/content/drive/MyDrive/ColabNotebooks')

In [ ]:
# os.getcwd()
os.chdir('/content/drive/MyDrive/ColabNotebooks/Aneurysm_Detection-main/dataset_creation')
%run create_dataset_patches_neg_and_pos.py --config config_creation_ds_patches.json

In [ ]:
!git clone https://github.com/MIC-DKFZ/nnUNet.git
%cd nnUNet
!pip install -e .
# nnUNetv2_plan_and_preprocess -d DATASET_ID --verify_dataset_integrity

In [1]:
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
# !pip3 install torchvision
# !pip3 install numpy==1.14.1

# Import all the necessary packages
import numpy as np
import nibabel as nib
# import itk
# import itkwidgets
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

from collections import defaultdict
import time
import os

from tqdm import tqdm

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn

import torchvision.utils as vutils

cudnn.benchmark = True
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
input_path = "/content/drive/MyDrive/ColabNotebooks/Aneurysm_Detection-main/dataset/derivatives/N4_bias_field_corrected/sub-000/ses-20110101/anat"
input_filename = os.path.join(input_path,'sub-000_ses-20110101_desc-angio_N4bfc_mask.nii.gz')
image_obj = nib.load(input_filename)
print(f'Type of the image {type(image_obj)}')

Type of the image <class 'nibabel.nifti1.Nifti1Image'>


In [ ]:
# Extract data as numpy ndarray
image_data = image_obj.get_fdata()
# Get the image shape and print it out
height, width, depth = image_data.shape
print(f"The image object has the following dimensions: height: {height}, width:{width}, depth:{depth}")
# Select random layer number
maxval = depth
i = np.random.randint(0, maxval)
# Define a channel to look at
channel = 0
print(f"Plotting Layer {i} Channel {channel} of Image")
plt.imshow(image_data[:, :, i], cmap='gray')
plt.axis('off');